In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.2'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,796 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 1,915 kB in 1s (1,480 kB/s)
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [6]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True, quote="\"", escape="\"")
df.createOrReplaceTempView('home_sales')



In [42]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""SELECT ROUND(mean(price),2)
            FROM home_sales
            WHERE bedrooms=4""").show()


+---------------------+
|round(mean(price), 2)|
+---------------------+
|            299661.01|
+---------------------+



In [43]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""SELECT ROUND(mean(price),2)
          FROM home_sales
          WHERE bedrooms=3 AND bathrooms=3
          GROUP BY date_built""").show()



+---------------------+
|round(mean(price), 2)|
+---------------------+
|             288770.3|
|            295962.27|
|            290852.27|
|            293683.19|
|            290555.07|
|            292859.62|
|            291117.47|
|            292676.79|
+---------------------+



In [44]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""SELECT ROUND(mean(price),2)
          FROM home_sales
          WHERE bedrooms=3 AND bathrooms=3 AND floors=2 AND sqft_living >= 2000
          GROUP BY date_built""").show()


+---------------------+
|round(mean(price), 2)|
+---------------------+
|            297609.97|
|            303676.79|
|            298264.72|
|            307539.97|
|             293965.1|
|            285010.22|
|            276553.81|
|            280317.58|
+---------------------+



In [45]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view, ROUND(mean(price),2)
          FROM home_sales
          GROUP BY view
          HAVING ROUND(mean(price),2) >= 350000""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
|  85|           1056336.74|
|  65|            736679.93|
|  53|             755214.8|
|  78|           1080649.37|
|  81|           1053472.79|
|  76|           1058802.78|
|  91|           1137372.73|
|  93|           1026006.06|
|  52|            733780.26|
|  86|           1070444.25|
|  94|            1033536.2|
|  57|             734340.5|
|  54|            798684.82|
|  96|           1017815.92|
|  92|            970402.55|
|  64|            767036.67|
|  61|            746877.59|
|  88|           1031719.35|
|  72|            780914.67|
|  59|             791453.0|
+----+---------------------+
only showing top 20 rows

--- 0.9476108551025391 seconds ---


In [46]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [47]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [48]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""SELECT view, ROUND(mean(price),2)
          FROM home_sales
          GROUP BY view
          HAVING ROUND(mean(price),2) >= 350000""").show()

print("--- %s seconds ---" % (time.time() - start_time)) #did it in about half the time


+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
|  85|           1056336.74|
|  65|            736679.93|
|  53|             755214.8|
|  78|           1080649.37|
|  81|           1053472.79|
|  76|           1058802.78|
|  91|           1137372.73|
|  93|           1026006.06|
|  52|            733780.26|
|  86|           1070444.25|
|  94|            1033536.2|
|  57|             734340.5|
|  54|            798684.82|
|  96|           1017815.92|
|  92|            970402.55|
|  64|            767036.67|
|  61|            746877.59|
|  88|           1031719.35|
|  72|            780914.67|
|  59|             791453.0|
+----+---------------------+
only showing top 20 rows

--- 0.47420740127563477 seconds ---


In [50]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.parquet('df_parquet', mode='overwrite')


In [51]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('df_parquet')

In [52]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView('parquet_temp_data')

In [53]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""SELECT view, ROUND(mean(price),2)
          FROM parquet_temp_data
          GROUP BY view
          HAVING ROUND(mean(price),2) >= 350000""").show()

print("--- %s seconds ---" % (time.time() - start_time)) #longer time for the parquet data

+----+---------------------+
|view|round(mean(price), 2)|
+----+---------------------+
|  85|           1056336.74|
|  65|            736679.93|
|  53|             755214.8|
|  78|           1080649.37|
|  81|           1053472.79|
|  76|           1058802.78|
|  91|           1137372.73|
|  93|           1026006.06|
|  52|            733780.26|
|  86|           1070444.25|
|  94|            1033536.2|
|  57|             734340.5|
|  54|            798684.82|
|  96|           1017815.92|
|  92|            970402.55|
|  64|            767036.67|
|  61|            746877.59|
|  88|           1031719.35|
|  72|            780914.67|
|  59|             791453.0|
+----+---------------------+
only showing top 20 rows

--- 1.5968530178070068 seconds ---


In [54]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [55]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False